In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import {
  createToken,
  Lexer,
  CstParser,
  IToken,
  CstNode,
  TokenType,
} from "chevrotain";
import * as fs from "fs";

# An Interpreter for a Simple Programming Language

In this notebook, we develop a complete interpreter for a small procedural programming language. The interpreter supports arithmetic expressions, boolean logic, control structures (`if`, `while`), blocks, and I/O operations (`read`, `print`).

## Architectural Overview

The interpreter operates in four distinct phases:

1.  **Lexical Analysis:** The source code is tokenized into a stream of tokens, filtering out comments and whitespace.
2.  **Parsing (CST Construction):** An **LL(k)** recursive-descent parser validates the syntax and builds a **Concrete Syntax Tree (CST)**.
3.  **Semantic Analysis & AST Generation:** A visitor traverses the CST to produce a clean **Abstract Syntax Tree (AST)**. This step resolves operator precedence and associativity logic.
4.  **Execution:** The AST is executed recursively. An **Environment** ($\sigma$) tracks variable states during execution.

## Grammar Specification

The grammar uses an iterative definition for expressions to accommodate the top-down parsing strategy.

$$
\begin{array}{lcl}
  \mathtt{program} & \rightarrow & \mathtt{stmnt}^* \\
  \mathtt{stmnt}   & \rightarrow & \mathtt{IF} \; \mathtt{'('} \; \mathtt{bool\_expr} \; \mathtt{')'} \; \mathtt{stmnt} \\
                   & \mid        & \mathtt{WHILE} \; \mathtt{'('} \; \mathtt{bool\_expr} \; \mathtt{')'} \; \mathtt{stmnt} \\
                   & \mid        & \mathtt{'\{'} \; \mathtt{stmnt}^* \; \mathtt{'\}'} \\
                   & \mid        & \mathtt{ID} \; \mathtt{':='} \; \mathtt{expr} \; \mathtt{';'} \\
                   & \mid        & \mathtt{expr} \; \mathtt{';'} \\[0.2cm]
  \mathtt{bool\_expr} & \rightarrow & \mathtt{expr} \; (\mathtt{'=='} \mid \mathtt{'!='} \mid \mathtt{'<='} \mid \mathtt{'>='} \mid \mathtt{'<'} \mid \mathtt{'>'}) \; \mathtt{expr} \\[0.2cm]
  \mathtt{expr}    & \rightarrow & \mathtt{product} \; \bigl( (\mathtt{'+'} \mid \mathtt{'-'}) \; \mathtt{product} \bigr)^* \\
  \mathtt{product} & \rightarrow & \mathtt{factor} \; \bigl( (\mathtt{'*'} \mid \mathtt{'/'} \mid \mathtt{'\%'}) \; \mathtt{factor} \bigr)^* \\
  \mathtt{factor}  & \rightarrow & \mathtt{'('} \; \mathtt{expr} \; \mathtt{')'} \\
                   & \mid        & \mathtt{NUMBER} \\
                   & \mid        & \mathtt{ID} \\
                   & \mid        & \mathtt{ID} \; \mathtt{'('} \; \mathtt{expr\_list}^? \; \mathtt{')'}
\end{array}
$$

## 1. Specification of the Scanner

We define the lexer tokens using regular expressions. We distinguish between keywords (which must be matched first), operators, and literals.

**Token Categories:**

* **Comments:** Support for block comments (`/* ... */`) and line comments (`// ...`). These are skipped.
* **Literals:** `NUMBER` (integers) and `ID` (identifiers).
* **Keywords:** `if`, `while`.
* **Operators:** Arithmetic (`+`, `-`, `*`, `/`, `%`), Relational (`==`, `!=`, `<=`, etc.), and Assignment (`:=`).
* **Structure:** Parentheses, Braces, Semicolons, Commas.

We allow both *single-line comments* and *multi-line comments*.
- The regular expression `/\*(.|\n)*?\*/` recognizes multi-line comments.
  Multi-line comments start with the string `/*` and end with the string `*/`.
  Note the use of the *non-greedy* quantor `*?`.  If we have code like
  ```
  /* blah */ a := 1; /* blub */
  ```
  the greedy quantor would recognize the whole line as one comment. 
- The regular expression `//.*` recognizes single-line comments.
  A single line comment starts with the string `//` and extends to the end of the line.

In [ ]:
const Comment: TokenType = createToken({
  name: "COMMENT",
  pattern: /\/\*[^]*?\*\/|\/\/[^\n]*/,
  group: Lexer.SKIPPED,
});

The token `NUMBER` specifies a natural number.

In [ ]:
const NumberTok: TokenType = createToken({
  name: "NUMBER",
  pattern: /0|[1-9][0-9]*/,
});

In [ ]:
const Assign: TokenType = createToken({ name: "ASSIGN", pattern: /:=/ });
const Eq: TokenType = createToken({ name: "EQ", pattern: /==/ });
const Ne: TokenType = createToken({ name: "NE", pattern: /!=/ });
const Le: TokenType = createToken({ name: "LE", pattern: /<=/ });
const Ge: TokenType = createToken({ name: "GE", pattern: />=/ });

The keywords 'int', 'if', 'else', 'while', 'return' have to be dealt with separately as they are syntactical identical to identifiers. The dictionary Keywords shown below maps every keyword to its token type.

In [ ]:
const IfTok: TokenType = createToken({ name: "IF", pattern: /if/ });
const WhileTok: TokenType = createToken({ name: "WHILE", pattern: /while/ });

When an identifier is read, we first have to check whether the identifier is one of our keywords.  If so, we assign the corresponding token type that is stored in the dictionary `Keywords`.  Otherwise, the token type is set to `ID`.

In [ ]:
const IdTok: TokenType = createToken({
  name: "ID",
  pattern: /[a-zA-Z][a-zA-Z0-9_]*/,
});

Operators consisting of a single character do not need an associated token type.
They are declared via the keyword `literals`.

In [ ]:
const Plus: TokenType = createToken({ name: "Plus", pattern: /\+/ });
const Minus: TokenType = createToken({ name: "Minus", pattern: /-/ });
const Mul: TokenType = createToken({ name: "Mul", pattern: /\*/ });
const Div: TokenType = createToken({ name: "Div", pattern: /\// });
const Mod: TokenType = createToken({ name: "Mod", pattern: /%/ });
const LParen: TokenType = createToken({ name: "LParen", pattern: /\(/ });
const RParen: TokenType = createToken({ name: "RParen", pattern: /\)/ });
const LBrace: TokenType = createToken({ name: "LBrace", pattern: /\{/ });
const RBrace: TokenType = createToken({ name: "RBrace", pattern: /\}/ });
const Semi: TokenType = createToken({ name: "Semi", pattern: /;/ });
const LT: TokenType = createToken({ name: "LT", pattern: /</ });
const GT: TokenType = createToken({ name: "GT", pattern: />/ });
const Comma: TokenType = createToken({ name: "Comma", pattern: /,/ });

Syntactically, newline characters are ignored. However, we still need to keep track of them in order to know the current line number, which is used for error messages. We achieve this by defining a `WhiteSpace` token that matches spaces, tabs, and line breaks but is skipped by the parser (`group: Lexer.SKIPPED`).

In [ ]:
const WhiteSpace: TokenType = createToken({
  name: "WhiteSpace",
  pattern: /[ \t\r\n]+/,
  group: Lexer.SKIPPED,
});

In [ ]:
const allTokens: TokenType[] = [
  WhiteSpace,
  Comment,
  Assign,
  Eq,
  Ne,
  Le,
  Ge,
  Plus,
  Minus,
  Mul,
  Div,
  Mod,
  LParen,
  RParen,
  LBrace,
  RBrace,
  Semi,
  LT,
  GT,
  Comma,
  IfTok,
  WhileTok,
  IdTok,
  NumberTok,
];

In [ ]:
const SLexer: Lexer = new Lexer(allTokens, { positionTracking: "full" });

In [ ]:
function findColumn(token: IToken): number {
  return token.startColumn ?? 0;
}

### Helper Function `testScanner`

Reads a file, tokenizes it, and prints the token stream for verification.

**Input:**
* `fileName`: Path to the source file ($\texttt{string}$).

**Output:**
* Prints the sequence of tokens (Type, Value, Line, Column) to the console.

In [ ]:
function testScanner(fileName: string): void {
  const program: string = fs.readFileSync(fileName, "utf-8");
  console.log(program);
  const result = SLexer.tokenize(program);

  if (result.errors.length > 0) {
    for (const err of result.errors) {
      console.error(
        `Illegal character at offset ${err.offset}: ${err.message}`
      );
    }
  }

  for (const t of result.tokens) {
    const line: number = t.startLine ?? 0;
    const col: number = t.startColumn ?? 0;
    const val: string | number =
      t.tokenType.name === "NUMBER" ? parseInt(t.image, 10) : `'${t.image}'`;
    console.log(`LexToken(${t.tokenType.name},${val},${line},${col})`);
  }
}

In [ ]:
testScanner('sum.sl')

In [ ]:
testScanner('factorial.sl')

## 2. Specification of the Parser

We define the parser class `SLParser` by extending `CstParser`. The grammar rules are implemented as methods within this class.

The parser constructs a **Concrete Syntax Tree (CST)**, which represents the hierarchical structure of the input program based on the grammar.

### The Grammar

Below is the grammar for our language in standard BNF notation.
**Note:** Since we use an LL(k) parser (Chevrotain), we implement recursive rules like `expr` and `product` iteratively (using `MANY`) to avoid left-recursion, but the logical structure remains the same.

```
program
    : /* epsilon */
    | stmnt program
    
stmnt 
    : IF '(' bool_expr ')' stmnt                 
    | WHILE '(' bool_expr ')' stmnt
    | '{' stmnt_list '}' 
    | ID ':=' expr ';'  
    | expr ';'       

bool_expr 
    : expr '==' expr     
    | expr '!=' expr     
    | expr '<=' expr     
    | expr '>=' expr     
    | expr '<'  expr      
    | expr '>'  expr     
 
expr: expr '+' product                 
    | expr '-' product
    | product
              
product
    : product '*' factor               
    | product '/' factor
    | product '%' factor 
    | factor

factor
    : '(' expr ')' 
    | NUMBER
    | ID
    | ID '(' expr_list ')'

expr_list
    :
    | ne_expr_list

ne_expr_list
    : expr
    | expr ',' ne_expr_list
```

In [ ]:
class SLParser extends CstParser {
  constructor() {
    super(allTokens);
    this.performSelfAnalysis();
  }

  public program = this.RULE("program", () => {
    this.SUBRULE(this.stmnt_list);
  });

  public stmnt_list = this.RULE("stmnt_list", () => {
    this.MANY(() => {
      this.SUBRULE(this.stmnt);
    });
  });

  public stmnt = this.RULE("stmnt", () => {
    this.OR([
      {
        ALT: () => {
          this.CONSUME(IfTok);
          this.CONSUME1(LParen);
          this.SUBRULE(this.boolexpr);
          this.CONSUME1(RParen);
          this.SUBRULE1(this.stmnt);
        },
      },
      {
        ALT: () => {
          this.CONSUME(WhileTok);
          this.CONSUME2(LParen);
          this.SUBRULE2(this.boolexpr);
          this.CONSUME2(RParen);
          this.SUBRULE2(this.stmnt);
        },
      },
      {
        ALT: () => {
          this.CONSUME(LBrace);
          this.SUBRULE(this.stmnt_list);
          this.CONSUME(RBrace);
        },
      },
      {
        ALT: () => {
          this.CONSUME(IdTok);
          this.CONSUME(Assign);
          this.SUBRULE3(this.expr);
          this.CONSUME(Semi);
        },
      },
      {
        ALT: () => {
          this.SUBRULE4(this.expr);
          this.CONSUME2(Semi);
        },
      },
    ]);
  });

  public boolexpr = this.RULE("boolexpr", () => {
    this.SUBRULE(this.expr);
    this.OR([
      { ALT: () => this.CONSUME(Eq) },
      { ALT: () => this.CONSUME(Ne) },
      { ALT: () => this.CONSUME(Le) },
      { ALT: () => this.CONSUME(Ge) },
      { ALT: () => this.CONSUME(LT) },
      { ALT: () => this.CONSUME(GT) },
    ]);
    this.SUBRULE2(this.expr);
  });

  public expr = this.RULE("expr", () => {
    this.SUBRULE(this.product);
    this.MANY(() => {
      this.OR([
        { ALT: () => { this.CONSUME(Plus); this.SUBRULE2(this.product); } },
        { ALT: () => { this.CONSUME(Minus); this.SUBRULE3(this.product); } },
      ]);
    });
  });

  public product = this.RULE("product", () => {
    this.SUBRULE(this.factor);
    this.MANY(() => {
      this.OR([
        { ALT: () => { this.CONSUME(Mul); this.SUBRULE2(this.factor); } },
        { ALT: () => { this.CONSUME(Div); this.SUBRULE3(this.factor); } },
        { ALT: () => { this.CONSUME(Mod); this.SUBRULE4(this.factor); } },
      ]);
    });
  });

  public factor = this.RULE("factor", () => {
    this.OR([
      {
        ALT: () => {
          this.CONSUME1(LParen);
          this.SUBRULE(this.expr);
          this.CONSUME1(RParen);
        },
      },
      {
        ALT: () => {
          this.CONSUME1(IdTok);
          this.CONSUME2(LParen);
          this.OPTION(() => {
            this.SUBRULE(this.expr_list);
          });
          this.CONSUME2(RParen);
        },
      },
      { ALT: () => this.CONSUME(NumberTok) },
      { ALT: () => this.CONSUME2(IdTok) },
    ]);
  });

  public expr_list = this.RULE("expr_list", () => {
    this.SUBRULE(this.expr);
    this.MANY(() => {
      this.CONSUME(Comma);
      this.SUBRULE2(this.expr);
    });
  });
}

const parser = new SLParser();

## 3. Strict CST Interfaces

To ensure type safety during the transformation phase, we define TypeScript interfaces reflecting the structure of the Concrete Syntax Tree generated by Chevrotain.

In [ ]:
interface ProgramCtx { stmnt_list: CstNode[]; }
interface StmntListCtx { stmnt?: CstNode[]; }
interface StmntCtx {
  IF?: IToken[]; LParen?: IToken[]; boolexpr?: CstNode[]; RParen?: IToken[]; stmnt?: CstNode[];
  WHILE?: IToken[]; LBrace?: IToken[]; stmnt_list?: CstNode[]; RBrace?: IToken[];
  ID?: IToken[]; ASSIGN?: IToken[]; expr?: CstNode[]; Semi?: IToken[];
}
interface BoolexprCtx { expr: CstNode[]; EQ?: IToken[]; NE?: IToken[]; LE?: IToken[]; GE?: IToken[]; LT?: IToken[]; GT?: IToken[]; }
interface ExprCtx { product: CstNode[]; Plus?: IToken[]; Minus?: IToken[]; }
interface ProductCtx { factor: CstNode[]; Mul?: IToken[]; Div?: IToken[]; Mod?: IToken[]; }
interface FactorCtx {
  LParen?: IToken[]; expr?: CstNode[]; RParen?: IToken[];
  ID?: IToken[]; expr_list?: CstNode[];
  NUMBER?: IToken[];
}
interface ExprListCtx { expr: CstNode[]; Comma?: IToken[]; }

## 4. Abstract Syntax Tree (AST) Definitions

We define a strongly-typed AST using **Discriminated Unions**. This tree represents the semantic structure of the program, abstracting away syntactic details like semicolons or parentheses.

**Nodes:**
* **`Expr`**: Expressions evaluating to a number (Literals, Variables, Binary Ops, Function Calls).
* **`BoolExpr`**: Expressions evaluating to a boolean (Comparisons).
* **`Stmt`**: Executable instructions (Assignments, Loops, Conditionals, Blocks).

In [ ]:
type BinaryOp = "+" | "-" | "*" | "/" | "%";
type BoolOp = "==" | "!=" | "<=" | ">=" | "<" | ">";

interface NumberLiteral { kind: "NumberLiteral"; value: number; }
interface VariableAccess { kind: "VariableAccess"; name: string; }
interface BinaryExpr { kind: "BinaryExpr"; op: BinaryOp; left: Expr; right: Expr; }
interface FunctionCall { kind: "FunctionCall"; name: string; args: Expr[]; }

type Expr = NumberLiteral | VariableAccess | BinaryExpr | FunctionCall;

interface BoolExpr { kind: "BoolExpr"; op: BoolOp; left: Expr; right: Expr; }

// --- Statements ---
interface AssignStmt { kind: "AssignStmt"; variable: string; value: Expr; }
interface IfStmt { kind: "IfStmt"; condition: BoolExpr; thenBranch: Stmt; }
interface WhileStmt { kind: "WhileStmt"; condition: BoolExpr; body: Stmt; }
interface BlockStmt { kind: "BlockStmt"; statements: Stmt[]; }
interface ExprStmt { kind: "ExprStmt"; expression: Expr; } // z.B. Funktionsaufrufe ohne Zuweisung

type Stmt = AssignStmt | IfStmt | WhileStmt | BlockStmt | ExprStmt;


## 5. Visitor: CST to AST Transformation

The `ToASTVisitor` transforms the raw CST into the simplified AST.

### Algorithm: Handling Operator Precedence and Associativity

Since the grammar defines `expr` and `product` iteratively (e.g., `term (op term)*`), we must ensure strict left-to-right associativity during AST construction.

**Algorithm Sketch (for `expr` and `product`):**

Let $T = [t_0, t_1, \dots, t_n]$ be the operand nodes.
Let $Ops = \{op_0, \dots, op_{n-1}\}$ be the operator tokens.

1.  **Collect & Sort:** Gather all operator tokens (e.g., `Plus` and `Minus`) and sort them by their textual position (`startOffset`). This reconstructs the original sequence.
2.  **Fold Left:** Initialize $result \leftarrow \text{visit}(t_0)$.
3.  **Iterate:** For $i$ from $0$ to $n-1$:
    * $op \leftarrow Ops[i]$
    * $rhs \leftarrow \text{visit}(t_{i+1})$
    * $result \leftarrow \mathtt{BinaryExpr}(op, result, rhs)$

**Input:**
A CST Context.

**Output:**
A typed AST Node (`Stmt`, `Expr`, or `BoolExpr`).

In [ ]:
const BaseCstVisitor = parser.getBaseCstVisitorConstructor();

class ToASTVisitor extends BaseCstVisitor {
  constructor() {
    super();
    this.validateVisitor();
  }

  public program(ctx: ProgramCtx): Stmt {
    return this.visit(ctx.stmnt_list[0]);
  }

  public stmnt_list(ctx: StmntListCtx): BlockStmt {
    const stmts = ctx.stmnt ? ctx.stmnt.map((s) => this.visit(s)) : [];
    return { kind: "BlockStmt", statements: stmts };
  }

  public stmnt(ctx: StmntCtx): Stmt {
    if (ctx.IF) {
      return {
        kind: "IfStmt",
        condition: this.visit(ctx.boolexpr![0]),
        thenBranch: this.visit(ctx.stmnt![0]),
      };
    }
    if (ctx.WHILE) {
      return {
        kind: "WhileStmt",
        condition: this.visit(ctx.boolexpr![0]),
        body: this.visit(ctx.stmnt![0]),
      };
    }
    if (ctx.LBrace) {
      return this.visit(ctx.stmnt_list![0]);
    }
    if (ctx.ASSIGN) {
      return {
        kind: "AssignStmt",
        variable: ctx.ID![0].image,
        value: this.visit(ctx.expr![0]),
      };
    }
    if (ctx.expr) {
      return {
        kind: "ExprStmt",
        expression: this.visit(ctx.expr[0]),
      };
    }
    throw new Error("Unexpected statement");
  }

  public boolexpr(ctx: BoolexprCtx): BoolExpr {
    let op: BoolOp;
    if (ctx.EQ) op = "==";
    else if (ctx.NE) op = "!=";
    else if (ctx.LE) op = "<=";
    else if (ctx.GE) op = ">=";
    else if (ctx.LT) op = "<";
    else if (ctx.GT) op = ">";
    else throw new Error("Unknown bool operator");

    return {
      kind: "BoolExpr",
      op,
      left: this.visit(ctx.expr[0]),
      right: this.visit(ctx.expr[1]),
    };
  }

  public expr(ctx: ExprCtx): Expr {
    let node: Expr = this.visit(ctx.product[0]);

    if (ctx.product.length > 1) {
      const pluses = ctx.Plus || [];
      const minuses = ctx.Minus || [];
      const allOps = [...pluses, ...minuses].sort((a, b) => a.startOffset - b.startOffset);

      for (let i = 1; i < ctx.product.length; i++) {
        const right = this.visit(ctx.product[i]);
        const opToken = allOps[i - 1];

        if (opToken.tokenType.name === "Plus") {
          node = { kind: "BinaryExpr", op: "+", left: node, right };
        } else {
          node = { kind: "BinaryExpr", op: "-", left: node, right };
        }
      }
    }
    return node;
  }

  public product(ctx: ProductCtx): Expr {
    let node: Expr = this.visit(ctx.factor[0]);

    if (ctx.factor.length > 1) {
      const muls = ctx.Mul || [];
      const divs = ctx.Div || [];
      const mods = ctx.Mod || [];
      const allOps = [...muls, ...divs, ...mods].sort((a, b) => a.startOffset - b.startOffset);

      for (let i = 1; i < ctx.factor.length; i++) {
        const right = this.visit(ctx.factor[i]);
        const opToken = allOps[i - 1];

        let op: BinaryOp = "*";
        if (opToken.tokenType.name === "Div") op = "/";
        else if (opToken.tokenType.name === "Mod") op = "%";

        node = { kind: "BinaryExpr", op, left: node, right };
      }
    }
    return node;
  }

  public factor(ctx: FactorCtx): Expr {
    // Case: '(' expr ')'
    if (ctx.LParen && ctx.expr && !ctx.ID) {
      return this.visit(ctx.expr[0]);
    }
    // Case: Function Call -> ID '(' expr_list? ')'
    if (ctx.ID && ctx.LParen) {
      const fname = ctx.ID[0].image;
      let args: Expr[] = [];
      if (ctx.expr_list) {
        // expr_list returns an array of Expr directly
        args = this.visit(ctx.expr_list[0]) as unknown as Expr[];
      }
      return { kind: "FunctionCall", name: fname, args };
    }
    // Case: Number
    if (ctx.NUMBER) {
      return { kind: "NumberLiteral", value: parseInt(ctx.NUMBER[0].image, 10) };
    }
    // Case: Variable
    if (ctx.ID) {
      return { kind: "VariableAccess", name: ctx.ID[0].image };
    }
    throw new Error("Unknown factor");
  }

  public expr_list(ctx: ExprListCtx): Expr[] {
    const exprs: Expr[] = [this.visit(ctx.expr[0])];
    if (ctx.expr.length > 1) {
      for (let i = 1; i < ctx.expr.length; i++) {
        exprs.push(this.visit(ctx.expr[i]));
      }
    }
    return exprs;
  }
}

const toAST = new ToASTVisitor();

## 6. Semantic Evaluation

We define the runtime semantics of the language.

**Components:**
* **`Env`**: The environment $\sigma$ mapping variable names to numerical values.
* **`InputProvider`**: An abstraction for reading input values (mocked here via `ArrayInput`).

In [ ]:
type NumberValue = number;
type Env = Record<string, NumberValue>;

interface InputProvider {
  read(): number;
}

class ArrayInput implements InputProvider {
  private readonly values: number[];
  private index = 0;
  constructor(values: number[]) { this.values = values; }
  read(): number {
    if (this.index >= this.values.length) throw new Error("No more input");
    return this.values[this.index++];
  }
}

### Function `evaluateExpr`

Evaluates arithmetic expressions.

**Input:**
* `expr`: The AST expression node.
* `env`: Current environment.
* `input`: Input provider.

**Output:**
* A number ($\mathbb{R}$).

In [ ]:
function evaluateExpr(expr: Expr, env: Env, input: InputProvider): number {
  switch (expr.kind) {
    case "NumberLiteral":
      return expr.value;
    case "VariableAccess":
      if (env[expr.name] === undefined) throw new Error(`Undefined var ${expr.name}`);
      return env[expr.name];
    case "BinaryExpr": {
      const l = evaluateExpr(expr.left, env, input);
      const r = evaluateExpr(expr.right, env, input);
      switch (expr.op) {
        case "+": return l + r;
        case "-": return l - r;
        case "*": return l * r;
        case "/": return l / r;
        case "%": return l % r;
      }
    }
    case "FunctionCall": {
      if (expr.name === "read") return input.read();
      throw new Error(`Unknown function ${expr.name}`);
    }
  }
}

### Function `evaluateBool`

Evaluates boolean comparison expressions.

**Input:**
* `expr`: The boolean AST node.
* `env`: Current environment.

**Output:**
* A boolean value (`true` / `false`).

In [ ]:
function evaluateBool(expr: BoolExpr, env: Env, input: InputProvider): boolean {
  const l = evaluateExpr(expr.left, env, input);
  const r = evaluateExpr(expr.right, env, input);
  switch (expr.op) {
    case "==": return l === r;
    case "!=": return l !== r;
    case "<=": return l <= r;
    case ">=": return l >= r;
    case "<": return l < r;
    case ">": return l > r;
  }
}

### Function `executeStmt`

Executes a statement, performing side effects on the environment or I/O.

**Input:**
* `stmt`: The AST statement node.
* `env`: The environment (mutable).
* `input`: Input provider.

**Semantics:**
* **`AssignStmt`**: Updates $\sigma[x] \leftarrow \text{eval}(e)$.
* **`IfStmt`**: Evaluates condition $c$. If true, executes the `thenBranch`.
* **`WhileStmt`**: Repeatedly executes `body` as long as condition $c$ evaluates to true.
* **`BlockStmt`**: Executes a list of statements sequentially.
* **`ExprStmt`**: Evaluates an expression. Handles the built-in `print()` function.

In [ ]:
function executeStmt(stmt: Stmt, env: Env, input: InputProvider): void {
  switch (stmt.kind) {
    case "BlockStmt":
      for (const s of stmt.statements) executeStmt(s, env, input);
      break;
    case "AssignStmt":
      env[stmt.variable] = evaluateExpr(stmt.value, env, input);
      break;
    case "ExprStmt":
      // Check for print specifically
      if (stmt.expression.kind === "FunctionCall" && stmt.expression.name === "print") {
        const val = evaluateExpr(stmt.expression.args[0], env, input);
        console.log(val);
      } else {
        evaluateExpr(stmt.expression, env, input);
      }
      break;
    case "IfStmt":
      if (evaluateBool(stmt.condition, env, input)) {
        executeStmt(stmt.thenBranch, env, input);
      }
      break;
    case "WhileStmt":
      while (evaluateBool(stmt.condition, env, input)) {
        executeStmt(stmt.body, env, input);
      }
      break;
  }
}

## 7. Main Execution Pipeline

We provide helper functions to tie the lexer, parser, visitor, and interpreter together.

In [ ]:
function parseProgram(source: string): Stmt {
  const lexRes = SLexer.tokenize(source);
  if (lexRes.errors.length > 0) throw new Error(lexRes.errors[0].message);
  
  parser.input = lexRes.tokens;
  const cst = parser.program();
  if (parser.errors.length > 0) throw new Error(parser.errors[0].message);
  
  return toAST.visit(cst) as Stmt;
}


### Function `runProgram`

**Input:**
* `fileName`: Path to the source code.
* `inputValues`: Array of numbers to serve as user input.

**Output:**
* The final state of the environment (`Env`).

**Process:**
1.  Read file $\rightarrow$ String.
2.  Tokenize $\rightarrow$ CST (via Parser).
3.  Transform $\rightarrow$ AST (via Visitor).
4.  Initialize empty Env $\sigma$.
5.  Execute AST with $\sigma$ and Input.

In [ ]:
function runProgram(fileName: string, inputValues: number[]): Env {
  const source = fs.readFileSync(fileName, "utf-8");
  const ast = parseProgram(source);
  const env: Env = {};
  const input = new ArrayInput(inputValues);
  
  executeStmt(ast, env, input);
  return env;
}

We execute the program `factorial.sl` with the input `5`.

In [ ]:
console.log("Starte factorial.sl (Strict Mode) mit Input 5...");
const resultfactorial = runProgram("factorial.sl", [5]);
console.log("Environment:", resultfactorial);

We execute the program `sum.sl` with the input `5`.

In [ ]:
console.log("Starte sum.sl (Strict Mode) mit Input 5...");
const resultsum = runProgram("sum.sl", [5]);
console.log("Environment:", resultsum);